In [42]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [155]:
class Value():
    def __init__(self,data,_children=(), _op=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None 
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data + other.data, (self,other),'+')
        def _backward():
            self.grad += 1.0*out.grad
            other.grad += 1.0*out.grad
        out._backward = _backward
        return out
    
    def __neg__(self):
        return -1*self
    
    def __sub__(self,other):
        return self + (-other)

    def __radd__(self,other):
        return self+other

    def __rsub__(self,other):
        return self-other
    
    def __mul__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data * other.data, (self,other),"*")
        def _backward():
            self.grad += other.data*out.grad
            other.grad += self.data*out.grad
        out._backward = _backward
        return out
    
    def __rmul__(self,other):
        return self*other
    
    def __pow__(self,other):
        assert isinstance(other,(int,float)), "Power can only be int or float"
        out = Value(self.data**other,(self,),f'**{other}')
        def _backward():
            self.grad += other*(self.data**(other-1))*out.grad
        out._backward = _backward
        return out
    def __truediv__(self,other):
        return self * other**-1
    def __rtruediv__(self,other):
        return self * other**-1
    
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x)-1)/(math.exp(2*x)+1)
        out = Value(t,(self,),'tanh')
        def _backwards():
            self.grad += (1 - t**2)*out.grad
        out._backward = _backwards
        return out
    
    def expo(self):
        x = self.data
        out = Value(math.exp(x),(self,),'exp')
        def _backwards():
            self.grad += out.data*out.grad
        out._backward = _backwards
        return out


    def backward(self):
        topo = []
        visited = set()
        def build_topo(node):
            if node not in visited:
                visited.add(node)
                for child in node._prev:
                    build_topo(child)
            topo.append(node)
        build_topo(self)
    
        self.grad = 1
        for node in topo[::-1]:
            node._backward()


In [156]:
x1 = Value(0)
w1 = Value(2)
x2 = Value(3)
w2 = Value(1)
b = Value(-2)

x1w1 = x1*w1
x2w2 = x2*w2

x1w1x2w2 = x1w1+x2w2

n=x1w1x2w2+b
print(n)
o=n.tanh()
o

Value(data=1)


Value(data=0.7615941559557649)

In [157]:
o.backward()

In [158]:
w2.grad

1.2599230248420783

In [159]:
w1/x2

Value(data=0.6666666666666666)

In [160]:
class Neuron:
    def __init__(self,nin):
        self.w = [Value(np.random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(np.random.uniform(-1,1))
    def __call__(self,x):
        act = self.b
        for wi, xi in zip(self.w,x):
            act += wi*xi
        out = act.tanh()
        return out
    def parameters(self):
        return self.w + [self.b]

In [161]:
class Layer:
    def __init__(self,nin,nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    def __call__(self, x):
        self.outs = [n(x) for n in self.neurons]
        return self.outs
    def parameters(self):
        params = []
        for neuron in self.neurons:
            ps = neuron.parameters()
            params.extend(ps)
        return params

In [162]:
class MLP:
    def __init__(self,nin: int,nout: list):
        sz = [nin]+nout
        self.layers = [Layer(sz[i],sz[i+1]) for i in range(len(nout))]
    def __call__(self,x):
        for layer in self.layers:
            x = layer(x)
        return x
    def parameters(self):
        params = []
        for layer in self.layers:
            ps = layer.parameters()
            params.extend(ps)
        return params
        

In [163]:
n = MLP(2,[6,6,1])
x = [2.0,3.0]
n(x)

[Value(data=0.619063536961183)]

In [164]:
xs = [
    [2.0,3.0,-1.0],
    [3.0,-1.0,0.5],
    [-0.5,1.0,1.0],
    [1.0,1.0,-1.0]
]

ys = [1.0,-1.0,-1.0,1.0]

y_pred = [n(x) for x in xs]
y_pred

[[Value(data=0.619063536961183)],
 [Value(data=0.5770782571215954)],
 [Value(data=0.8229916296568681)],
 [Value(data=0.5540421550675353)]]

In [167]:
loss = sum([(y_out[0]-y_gt)**2 for y_gt, y_out in zip(ys,y_pred)])
loss

Value(data=6.154465299214025)

In [151]:
loss.backward()

In [152]:
n.layers[0].neurons[0].w[0].grad

-4.144115527738787

In [153]:
n = MLP(2,[3,3,1])

for k in range(100):
    y_pred = [n(x) for x in xs]
    loss = sum([(y_out[0]-y_gt)**2 for y_gt, y_out in zip(ys,y_pred)])
    for p in n.parameters():
        p.grad = 0
    loss.backward()
    for p in n.parameters():
        p.data += -0.1 * p.grad
    print(k,loss.data)

0 4.975122772555551
1 3.55305032975972
2 2.7304789319151346
3 3.2893711925236455
4 2.7312368489187895
5 2.402979517308912
6 1.5203235988438772
7 2.287529603287638
8 3.4446445113228856
9 1.2723577290818076
10 1.648139220437081
11 3.5325541211190528
12 0.36235570013920193
13 0.19238038769802746
14 0.1467778509633363
15 0.11814548240011431
16 0.09896799740732573
17 0.08599086162865391
18 0.07544524813417687
19 0.06752425646075655
20 0.0609211119969099
21 0.055455213342794005
22 0.050938248561463714
23 0.04710755197649699
24 0.04378339801266237
25 0.04078401385634208
26 0.03819933977896019
27 0.03586344669214102
28 0.033768101032954786
29 0.03195358933998554
30 0.03026657223708513
31 0.028771885151291456
32 0.02741769834573055
33 0.026145880250917467
34 0.024980222292506443
35 0.02390757512727952
36 0.022942055216708078
37 0.02202534748273192
38 0.021175371429981
39 0.02039813169153088
40 0.019661316248561374
41 0.018973559972636205
42 0.018328253092745085
43 0.0177325747876818
44 0.017177

In [154]:
y_pred

[[Value(data=0.9684279602272388)],
 [Value(data=-0.9657066308767059)],
 [Value(data=-0.954075660064176)],
 [Value(data=0.9587768607315286)]]